<a href="https://colab.research.google.com/github/AliTavakoli2001/Deep-Learning-Project/blob/main/Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Add the necessary Frameworks and Libraries**





In [ ]:
import tensorflow as TF
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.datasets import mnist
import numpy as NP
import matplotlib.pyplot as plt

---
# **Load Mnist Data**


> **Why unlabeled data?**

In an autoencoder, we only need unlabeled data because it is an unsupervised learning model. The goal of an autoencoder is to learn the underlying patterns of the data and reconstruct the input data without requiring any labels. It focuses on minimizing the reconstruction error rather than predicting labels.







In [ ]:
(x_train, _), (x_test, _) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step




---


# **Normalize data to range [0,1]**


> **Normalizing Image Data for Better Model Performance**

The two lines of code normalize the image data by converting pixel values from the range [0, 255] to [0, 1]. This is done by first converting the data type to float32 and then dividing by 255. This normalization helps improve model performance and training stability by ensuring the input features are on a similar scale.



In [ ]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


---

# Flatten images

The `reshape` function converts each 2D image into a 1D vectorby flattening the pixel values row-wise. This transformation is necessary because fully connected neural networks expect 1D input vectors instead of 2D matrices.

In [ ]:
x_train = x_train.reshape((len(x_train), -1))
x_test = x_test.reshape((len(x_test), -1))



---


# Define Autoencoder model


1. The input layer `Input(shape=(784,))` takes a **flattened 28×28 image** as a **1D vector** with **784 values**.  
2. The **encoder layer** `Dense(32, activation='relu')` reduces the input **from 784 to 32 dimensions**, extracting key features.  
3. The **decoder layer** `Dense(784, activation='sigmoid')` reconstructs the original **784-dimensional** image from the **compressed 32-dimensional** representation.  
4. **ReLU** helps retain important features in encoding, while **sigmoid** ensures output values remain between **0 and 1**, matching the normalized pixel range.  

In [ ]:
encoding_dim = 32

input_img = Input(shape=(784,))
encoded = Dense(encoding_dim, activation='relu')(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)



---

# model summary

This will show the architecture and the number of parameters

In [ ]:
autoencoder.summary()



---


# Compile model

This line compiles the autoencoder using the **Adam optimizer** for efficient and adaptive learning and **binary cross-entropy loss**, which is suitable for pixel values (0 to 1) and helps minimize reconstruction error.

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



---


# Train the autoencoder
This line trains the autoencoder model for 50 epochs using the training data (x_train) as both input and target. It processes data in batches of 256, shuffling the data before each epoch. The validation data (x_test) is used to track performance during training, and a progress bar is shown during training with verbose=1. The training history, including loss and accuracy, is saved in the history variable.

In [ ]:
history = autoencoder.fit(x_train, x_train, epochs=50, batch_size=256, shuffle=True, validation_data=(x_test, x_test), verbose=1)



---


# Visualizing the results

This function displays images from the **original test data** and the **reconstructed images** generated by the autoencoder model in a two-row plot. The first row shows the original images, and the second row shows the reconstructed images from the model. This helps you evaluate the model's performance in terms of image reconstruction.

In [ ]:
def plot_results(x_test, decoded_imgs, n=10):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
        plt.axis('off')

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
    plt.show()



---


# Encode and decode some test images

This line of code uses the `autoencoder` model to make **predictions** on the input test data (`x_test`). It generates **reconstructed images** that should closely resemble the original test images, though with some potential minor loss or distortion. The reconstructed images are stored in the `decoded_imgs` variable.

In [ ]:
decoded_imgs = autoencoder.predict(x_test)



---


# Plot the results

In [ ]:
plot_results(x_test, decoded_imgs)




---


# Plot training & validation accuracy

This code plots the **model's accuracy** over training and validation datasets throughout the epochs. It visualizes the accuracy on the training data (`Train`) and the validation data (`Test`) over the course of training, with labeled axes for epochs and accuracy. The chart helps track the model's performance and improvement during training.

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()